<a href="https://colab.research.google.com/github/jus-tinian/vix_calc/blob/master/KoalasTD_SPXquotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create Working Env

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
!pip install -q findspark
!pip install -q pyspark

In [0]:
!pip install -q koalas

## Imports

In [0]:
import requests
import datetime

import pyspark
import databricks.koalas as ks

from pyspark.sql import SparkSession

## Create Spark Session

In [0]:
APP_NAME = "VIXcalc"
SPARK_URL = "local[*]"

In [0]:
spark = SparkSession.builder.master(SPARK_URL).appName(APP_NAME).getOrCreate()

In [9]:
spark

## Get API response

In [0]:
today = datetime.datetime.today().date()

In [0]:
nearDate = today + datetime.timedelta(days=23)

In [0]:
farDate = today + datetime.timedelta(days=37)

In [0]:
# your key here
KEY = 'your key'

In [0]:
req = requests.get(f"https://api.tdameritrade.com/v1/marketdata/chains?apikey={KEY}&symbol=%24SPX.X&includeQuotes=TRUE&strategy=SINGLE&range=ALL&fromDate={str(nearDate)}&toDate={str(farDate)}")

## Set Interest Rate

In [0]:
RATE = req.json()['interestRate']

## Prep Pipe

In [0]:
def from_json(jsonObj, sideKey, termKey):

  # had to set to avoid error in if; elif
  ks.set_option('compute.ops_on_diff_frames', True)

  source_df1 = ks.DataFrame(
      {
       k:v for k,v in jsonObj[sideKey][termKey][strike][0].items() if k in
       ['strikePrice', 'symbol', 'putCall', 'bid', 'ask', 'expirationDate', 'inTheMoney']
      }
       for strike in jsonObj[sideKey][termKey].keys()
      )

  source_df2 = source_df1.loc[source_df1['bid'] != 0, :]

  standardCount = source_df2.loc[~source_df2['symbol'].str.contains('W'), :].shape[0]

  # set to select standard expirations and adds time to expiration date
  if standardCount != 0:
    source_df2 = source_df2.loc[~source_df2['symbol'].str.contains('W'), :]
    source_df2['expiration'] = ks.to_datetime(termKey[:-3]) + datetime.timedelta(hours=9.5)
  
  # handle weeklies if weeklies only
  elif standardCount == 0:
    source_df2['expiration'] = ks.to_datetime(termKey[:-3]) + datetime.timedelta(hours=16)

  source_df2 = source_df2.rename(columns={'expiration': 'expirationDate', 'expirationDate': 'expirationEpoch'})

  return source_df2

In [0]:
def combineContracts(jsonObj, term='near'):

  dateKeys = [i for i in list(jsonObj['putExpDateMap'].keys()) if datetime.datetime.strptime(i[:-3], '%Y-%m-%d').weekday() == 4]
  termKey = min(dateKeys) if term.lower() == 'near' else max(dateKeys)

  return from_json(jsonObj, 'putExpDateMap', termKey).append(from_json(jsonObj, 'callExpDateMap', termKey)).sort_values(by='strikePrice').set_index('strikePrice')

# Near

In [0]:
near = combineContracts(req.json(), 'near')

In [24]:
near.head()

,putCall,symbol,bid,ask,expirationEpoch,inTheMoney,expirationDate
strikePrice,,,,,,,
500.0,CALL,SPXW_061220C500,2443.2,2450.1,1591992000000,True,2020-06-12 16:00:00
600.0,CALL,SPXW_061220C600,2343.3,2350.2,1591992000000,True,2020-06-12 16:00:00
700.0,CALL,SPXW_061220C700,2243.3,2250.2,1591992000000,True,2020-06-12 16:00:00
800.0,CALL,SPXW_061220C800,2144.2,2150.2,1591992000000,True,2020-06-12 16:00:00
900.0,CALL,SPXW_061220C900,2044.2,2050.3,1591992000000,True,2020-06-12 16:00:00


## Far

In [0]:
far = combineContracts(req.json(), 'far')

In [26]:
far.head()

,putCall,symbol,bid,ask,expirationEpoch,inTheMoney,expirationDate
strikePrice,,,,,,,
100.0,CALL,SPX_061920C100,2844.6,2848.4,1592596800000,True,2020-06-19 09:30:00
200.0,CALL,SPX_061920C200,2744.7,2748.4,1592596800000,True,2020-06-19 09:30:00
300.0,CALL,SPX_061920C300,2644.7,2648.5,1592596800000,True,2020-06-19 09:30:00
400.0,CALL,SPX_061920C400,2544.7,2548.5,1592596800000,True,2020-06-19 09:30:00
500.0,CALL,SPX_061920C500,2444.8,2448.5,1592596800000,True,2020-06-19 09:30:00
